In [6]:
import statsapi as mlb
import pandas as pd 
from Humble_Chuck import mysql_helper1 as sql
import sys

In [15]:
sys.path[0] = '/Users/schlinkertc/Flatiron/projects/MLB_Stats/'

In [16]:
import config
import charlie_functions as c 

In [18]:
flatiron = sql.Connection(config.host,config.admin,config.password)
MLB_Stats = sql.DataBase(flatiron,'MLB_Stats')

In [283]:
games_query = MLB_Stats.query_to_df("""select gameid,game_date from games where year(game_date)=2019""")

In [290]:
gameids = list(games_query['gameid'][::70])

In [291]:
gameids

[564734,
 564805,
 564893,
 564981,
 565069,
 565157,
 565243,
 565329,
 565417,
 565489,
 565577,
 565663,
 565750,
 565836,
 565921,
 566009,
 566093,
 566163,
 566248,
 566332,
 566418,
 566506,
 566593,
 566681,
 566769,
 566846,
 566931,
 567019,
 567105,
 567202,
 567291,
 567379,
 567465,
 567555,
 567625]

In [128]:
#c.get_gameData(gameids)

In [132]:
def APIcall(gamePks):
    #get info on the game from statsapi
    results = []
    for gamePk in gamePks: 
        result = mlb.get('game',{"gamePk":gamePk})
        results.append(result)
    return results
        

In [292]:
results = APIcall(gameids)

In [293]:
def get_gameData(API_results):
    #takes in a list of results from APIcall function
    #list of nested dicts for each game
    results = []
    for result in API_results:
        
        result = result['gameData']
        
        results.append(result)
    
    return results

In [294]:
gameData = get_gameData(results)

In [295]:
def games_table(API_results):
    results = get_gameData(API_results)
    
    games = []
    for result in results:

        #this is a list gameData's keys that do not have any further nested dictionaries
        #these are ready to be put into a table together and identified by the gamePk
        gameData_keys = ['game','datetime','status','flags','weather']

        game = result['game']
        for x in gameData_keys[1:]:
            game.update(result[x])
        
        game['away_id']=result['teams']['away']['id']
        game['home_id']=result['teams']['home']['id']
        
        game['venue_id']=result['venue']['id']
        games.append(game)
    return pd.DataFrame(games)

In [296]:
games = games_table(results)

In [297]:
games.head()

,pk,type,doubleHeader,id,gamedayType,tiebreaker,gameNumber,calendarEventID,season,seasonDisplay,...,statusCode,abstractGameCode,noHitter,perfectGame,condition,temp,wind,away_id,home_id,venue_id
0,564734,R,N,2019/04/04/texmlb-anamlb-1,P,N,1,14-564734-2019-04-04,2019,2019,...,F,F,False,False,Overcast,61,"6 mph, Out To LF",140,108,1
1,564805,R,N,2019/09/11/clemlb-anamlb-1,P,N,1,14-564805-2019-09-11,2019,2019,...,F,F,False,False,Clear,79,"8 mph, Out To CF",114,108,1
2,564893,R,N,2019/08/20/colmlb-arimlb-1,P,N,1,14-564893-2019-08-20,2019,2019,...,F,F,False,False,Roof Closed,75,"0 mph, None",115,109,15
3,564981,R,N,2019/08/01/tormlb-balmlb-1,P,N,1,14-564981-2019-08-01,2019,2019,...,F,F,False,False,Partly Cloudy,86,"6 mph, In From RF",141,110,2
4,565069,R,N,2019/07/12/lanmlb-bosmlb-1,P,N,1,14-565069-2019-07-12,2019,2019,...,F,F,False,False,Partly Cloudy,73,"5 mph, R To L",119,111,3


In [259]:
results = APIcall(gameids)

In [300]:
def teams_table(API_results):
    results = get_gameData(API_results)
    
    teams=[]
    teams_record = []
    for result in results:
        team = result['teams']['home']
        
        team_record = team.pop('record')
        team_record['team_id'] = result['teams']['home']['id']
        team_record['gamePk'] = result['game']['pk']
        
        for x in list(team.keys()):
            if type(team[x])==dict:
                team[x] = team[x].get('id','null')

        teams.append(team)
    
    teams = pd.DataFrame(teams)
    return teams.drop_duplicates(subset='id')

In [301]:
results = APIcall(gameids)
teams = teams_table(results)

In [302]:
teams

,id,name,link,season,venue,teamCode,fileCode,abbreviation,teamName,locationName,firstYearOfPlay,league,division,sport,shortName,springLeague,allStarStatus,active
0,108,Los Angeles Angels,/api/v1/teams/108,2019,1,ana,ana,LAA,Angels,Anaheim,1961,103,200,1,LA Angels,114,N,True
2,109,Arizona Diamondbacks,/api/v1/teams/109,2019,15,ari,ari,ARI,D-backs,Phoenix,1998,104,203,1,Arizona,114,N,True
3,110,Baltimore Orioles,/api/v1/teams/110,2019,2,bal,bal,BAL,Orioles,Baltimore,1901,103,201,1,Baltimore,115,N,True
4,111,Boston Red Sox,/api/v1/teams/111,2019,3,bos,bos,BOS,Red Sox,Boston,1901,103,201,1,Boston,115,N,True
5,112,Chicago Cubs,/api/v1/teams/112,2019,17,chn,chc,CHC,Cubs,Chicago,1874,104,205,1,Chi Cubs,114,N,True
6,113,Cincinnati Reds,/api/v1/teams/113,2019,2602,cin,cin,CIN,Reds,Cincinnati,1882,104,205,1,Cincinnati,114,N,True
7,114,Cleveland Indians,/api/v1/teams/114,2019,5,cle,cle,CLE,Indians,Cleveland,1901,103,202,1,Cleveland,114,N,True
8,115,Colorado Rockies,/api/v1/teams/115,2019,19,col,col,COL,Rockies,Denver,1992,104,203,1,Colorado,114,N,True
10,116,Detroit Tigers,/api/v1/teams/116,2019,2394,det,det,DET,Tigers,Detroit,1901,103,202,1,Detroit,115,N,True
11,117,Houston Astros,/api/v1/teams/117,2019,2392,hou,hou,HOU,Astros,Houston,1962,103,200,1,Houston,115,N,True


In [324]:
def venues_table(API_results):
    results = get_gameData(API_results)
    
    venues = []
    
    for result in results: 
        venue = result['venue']
        
#         for x in list(venue.keys()):
#             if type(venue[x])==dict: 
#                 venue.update(venue.pop(x))
        
#         for x in list(venue.keys()):
#             if type(venue[x])==dict: 
#                 venue.update(venue.pop(x))
        
        venues.append(venue)
    
    return venues

In [325]:
results = APIcall(gameids)
venues = venues_table(results)

In [326]:
venues[0]

{'id': 1,
 'name': 'Angel Stadium',
 'link': '/api/v1/venues/1',
 'location': {'city': 'Anaheim',
  'state': 'California',
  'stateAbbrev': 'CA',
  'defaultCoordinates': {'latitude': 33.80019044, 'longitude': -117.8823996}},
 'timeZone': {'id': 'America/Los_Angeles', 'offset': -8, 'tz': 'PST'},
 'fieldInfo': {'capacity': 45517,
  'turfType': 'Grass',
  'roofType': 'Open',
  'leftLine': 330,
  'left': 347,
  'leftCenter': 389,
  'center': 396,
  'rightCenter': 365,
  'right': 348,
  'rightLine': 330}}